In [ ]:
pip install networkx

In [ ]:
class Vuelo:
    def __init__(self, source_airport_code, source_airport_name, source_airport_city, source_airport_country,
                 source_airport_latitude, source_airport_longitude, destination_airport_code,
                 destination_airport_name, destination_airport_city, destination_airport_country,
                 destination_airport_latitude, destination_airport_longitude):
        self.source_airport_code = source_airport_code
        self.source_airport_name = source_airport_name
        self.source_airport_city = source_airport_city
        self.source_airport_country = source_airport_country
        self.source_airport_latitude = source_airport_latitude
        self.source_airport_longitude = source_airport_longitude
        self.destination_airport_code = destination_airport_code
        self.destination_airport_name = destination_airport_name
        self.destination_airport_city = destination_airport_city
        self.destination_airport_country = destination_airport_country
        self.destination_airport_latitude = destination_airport_latitude
        self.destination_airport_longitude = destination_airport_longitude


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import networkx as nx
import math
import csv
import heapq
from typing import List, Tuple, Dict

# Constantes
RADIO_TIERRA_KM = 6371  # Radio de la Tierra en km
RUTA_ARCHIVO_CSV = '/content/flights_final.csv'
RUTA_SALIDA_MAPA = 'mapa_aeropuertos.png'

# Estructura para almacenar información detallada de los aeropuertos
InfoAeropuerto = Dict[str, str]

# Tipos personalizados
Coordenada = Tuple[float, float]
Grafo = List[List[Tuple[int, float]]]

def distancia_haversine(coord1: Coordenada, coord2: Coordenada) -> float:
    """Calcula la distancia Haversine entre dos puntos en la Tierra."""
    lat1, lon1 = map(math.radians, coord1)
    lat2, lon2 = map(math.radians, coord2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return RADIO_TIERRA_KM * c

def cargar_datos_aeropuertos(ruta_archivo: str) -> Tuple[List[str], List[Coordenada], Grafo, List[InfoAeropuerto]]:
    """Carga los datos de aeropuertos desde un archivo CSV y construye el grafo."""
    codigos_aeropuertos = []
    coordenadas_aeropuertos = []
    info_aeropuertos = []
    grafo: Grafo = []

    def obtener_indice_aeropuerto(codigo: str) -> int:
        """Devuelve el índice de un aeropuerto o lo agrega si no existe."""
        if codigo not in codigos_aeropuertos:
            codigos_aeropuertos.append(codigo)
            coordenadas_aeropuertos.append((0, 0))  # Placeholder
            info_aeropuertos.append({})  # Inicializa diccionario vacío
            grafo.append([])
        return codigos_aeropuertos.index(codigo)

    with open(ruta_archivo, 'r') as archivo_csv:
        lector = csv.DictReader(archivo_csv)
        for fila in lector:
            codigo_origen = fila['Source Airport Code']
            lat_origen = float(fila['Source Airport Latitude'])
            lon_origen = float(fila['Source Airport Longitude'])
            codigo_destino = fila['Destination Airport Code']
            lat_destino = float(fila['Destination Airport Latitude'])
            lon_destino = float(fila['Destination Airport Longitude'])

            indice_origen = obtener_indice_aeropuerto(codigo_origen)
            indice_destino = obtener_indice_aeropuerto(codigo_destino)

            # Actualizar coordenadas
            coordenadas_aeropuertos[indice_origen] = (lat_origen, lon_origen)
            coordenadas_aeropuertos[indice_destino] = (lat_destino, lon_destino)

            # Guardar la información del aeropuerto
            info_aeropuertos[indice_origen] = {
                'codigo': codigo_origen,
                'nombre': fila['Source Airport Name'],
                'ciudad': fila['Source Airport City'],
                'pais': fila['Source Airport Country'],
                'latitud': str(lat_origen),
                'longitud': str(lon_origen)
            }
            info_aeropuertos[indice_destino] = {
                'codigo': codigo_destino,
                'nombre': fila['Destination Airport Name'],
                'ciudad': fila['Destination Airport City'],
                'pais': fila['Destination Airport Country'],
                'latitud': str(lat_destino),
                'longitud': str(lon_destino)
            }

            # Calcular la distancia y agregar al grafo
            distancia = distancia_haversine((lat_origen, lon_origen), (lat_destino, lon_destino))
            grafo[indice_origen].append((indice_destino, distancia))
            grafo[indice_destino].append((indice_origen, distancia))

    return codigos_aeropuertos, coordenadas_aeropuertos, grafo, info_aeropuertos

def dijkstra(grafo: Grafo, inicio: int) -> Tuple[List[float], List[int]]:
    """
    Algoritmo de Dijkstra para encontrar caminos mínimos desde un vértice.

    Args:
    grafo (Grafo): Grafo representado como lista de adyacencias.
    inicio (int): Índice del vértice inicial.

    Returns:
    Tuple[List[float], List[int]]: Distancias mínimas y predecesores.
    """
    n = len(grafo)
    distancias = [float('inf')] * n  # Inicializa las distancias a infinito
    predecesores = [-1] * n  # Inicializa los predecesores a -1
    distancias[inicio] = 0  # La distancia al nodo inicial es 0
    cola_prioridad = [(0, inicio)]  # Cola de prioridad con la distancia inicial

    while cola_prioridad:
        dist_actual, v_actual = heapq.heappop(cola_prioridad)  # Nodo con menor distancia actual

        # Si la distancia actual es mayor que la almacenada, significa que ya fue procesado
        if dist_actual > distancias[v_actual]:
            continue

        # Relajación de las aristas
        for vecino, peso in grafo[v_actual]:
            nueva_distancia = dist_actual + peso
            # Si encontramos un camino más corto a este vecino
            if nueva_distancia < distancias[vecino]:
                distancias[vecino] = nueva_distancia  # Actualiza la distancia mínima
                predecesores[vecino] = v_actual  # Actualiza el predecesor
                heapq.heappush(cola_prioridad, (nueva_distancia, vecino))  # Añadir a la cola de prioridad

    return distancias, predecesores

def mostrar_aeropuertos_lejanos_misma_componente(codigo: str, codigos_aeropuertos: List[str], info_aeropuertos: List[InfoAeropuerto], grafo: Grafo) -> None:
    """
    Muestra la información de un aeropuerto dado y los 10 aeropuertos más lejanos en la misma componente conexa.
    """
    if codigo not in codigos_aeropuertos:
        print(f"El aeropuerto con código {codigo} no se encuentra en la base de datos.")
        return

    indice_inicio = codigos_aeropuertos.index(codigo)
    info_inicio = info_aeropuertos[indice_inicio]

    print("Información del aeropuerto de inicio:")
    print(f"Código: {info_inicio['codigo']}")
    print(f"Nombre: {info_inicio['nombre']}")
    print(f"Ciudad: {info_inicio['ciudad']}")
    print(f"País: {info_inicio['pais']}")
    print(f"Latitud: {info_inicio['latitud']}")
    print(f"Longitud: {info_inicio['longitud']}")
    print("\n")

    # Encontrar la componente conexa del aeropuerto de inicio
    componentes = analizar_conectividad(grafo)
    componente_inicio = next(comp for comp in componentes if indice_inicio in comp)

    # Calcular las distancias mínimas desde el aeropuerto de inicio
    distancias, _ = dijkstra(grafo, indice_inicio)

    # Filtrar y ordenar los aeropuertos de la misma componente por distancia
    aeropuertos_componente = [(i, d) for i, d in enumerate(distancias) if i in componente_inicio and i != indice_inicio]
    aeropuertos_ordenados = sorted(aeropuertos_componente, key=lambda x: x[1], reverse=True)

    print("Los 10 aeropuertos más lejanos en la misma componente conexa:")
    for i, (indice, distancia) in enumerate(aeropuertos_ordenados[:10], 1):
        info = info_aeropuertos[indice]
        print(f"{i}. {info['nombre']} ({info['codigo']}):")
        print(f"   Ciudad: {info['ciudad']}, País: {info['pais']}")
        print(f"   Distancia: {distancia:.2f} km")
        print(f"   Latitud: {info['latitud']}, Longitud: {info['longitud']}")
        print()

def reconstruir_camino(predecesores: List[int], inicio: int, fin: int) -> List[int]:
    """Reconstruye el camino mínimo utilizando la lista de predecesores."""
    camino = []
    actual = fin
    while actual != -1:
        camino.append(actual)
        actual = predecesores[actual]
    return camino[::-1]

def visualizar_camino(camino: List[int], info_aeropuertos: List[InfoAeropuerto], grafo: Grafo) -> None:
    """Visualiza el camino mínimo entre dos aeropuertos con información detallada."""
    G = nx.Graph()

    # Crear figura y ejes
    fig, ax = plt.subplots(figsize=(20, 15))

    # Agregar nodos y aristas
    for i in range(len(camino)):
        info = info_aeropuertos[camino[i]]
        G.add_node(camino[i], pos=(float(info['longitud']), float(info['latitud'])), info=info)
        if i < len(camino) - 1:
            siguiente = camino[i+1]
            distancia = next(peso for vecino, peso in grafo[camino[i]] if vecino == siguiente)
            G.add_edge(camino[i], siguiente, weight=distancia)

    # Obtener posiciones de los nodos
    pos = nx.get_node_attributes(G, 'pos')

    # Dibujar aristas
    nx.draw_networkx_edges(G, pos, ax=ax, edge_color='gray')

    # Dibujar nodos personalizados
    for node, (x, y) in pos.items():
        info = G.nodes[node]['info']

        # Crear un círculo para el nodo
        circle = patches.Circle((x, y), 0.5, fill=True, facecolor='lightblue', edgecolor='black', linewidth=2, alpha=0.7)
        ax.add_patch(circle)

        # Agregar texto dentro del círculo
        texto = f"Código: {info['codigo']}\n" \
                f"Nombre: {info['nombre'][:20]}...\n" \
                f"Ciudad: {info['ciudad']}\n" \
                f"País: {info['pais']}\n" \
                f"Lat: {info['latitud']}\n" \
                f"Lon: {info['longitud']}"
        ax.text(x, y, texto, ha='center', va='center', fontsize=8, wrap=True)

    # Agregar etiquetas de peso a las aristas
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

    # Configurar límites y aspecto del gráfico
    ax.set_xlim(min(pos[node][0] for node in G.nodes) - 1, max(pos[node][0] for node in G.nodes) + 1)
    ax.set_ylim(min(pos[node][1] for node in G.nodes) - 1, max(pos[node][1] for node in G.nodes) + 1)
    ax.set_aspect('equal')

    plt.title("Camino mínimo entre aeropuertos", fontsize=16)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig('camino_minimo_detallado.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("Visualización detallada del camino guardada como 'camino_minimo_detallado.png'")

def mostrar_camino_minimo(codigo_inicio: str, codigo_fin: str, codigos_aeropuertos: List[str],
                          info_aeropuertos: List[InfoAeropuerto], grafo: Grafo) -> None:
    """Muestra el camino mínimo entre dos aeropuertos y detalles de cada tramo."""
    if codigo_inicio not in codigos_aeropuertos or codigo_fin not in codigos_aeropuertos:
        print("Uno o ambos códigos de aeropuerto no se encuentran en la base de datos.")
        return

    indice_inicio = codigos_aeropuertos.index(codigo_inicio)
    indice_fin = codigos_aeropuertos.index(codigo_fin)

    distancias, predecesores = dijkstra(grafo, indice_inicio)

    if distancias[indice_fin] == float('inf'):
        print(f"No existe un camino entre {codigo_inicio} y {codigo_fin}.")
        return

    camino = reconstruir_camino(predecesores, indice_inicio, indice_fin)

    print(f"Camino mínimo de {codigo_inicio} a {codigo_fin}:")
    print(f"Distancia total: {distancias[indice_fin]:.2f} km")
    print("\nDetalle del camino:")

    for i, indice in enumerate(camino):
        info = info_aeropuertos[indice]
        print(f"{i+1}. {info['nombre']} ({info['codigo']}):")
        print(f"   Ciudad: {info['ciudad']}, País: {info['pais']}")
        print(f"   Latitud: {info['latitud']}, Longitud: {info['longitud']}")

        if i < len(camino) - 1:
            siguiente = camino[i+1]
            distancia_tramo = next(peso for vecino, peso in grafo[indice] if vecino == siguiente)
            print(f"   Distancia al siguiente aeropuerto: {distancia_tramo:.2f} km")
        print()

    # Visualizar el camino con información detallada
    visualizar_camino(camino, info_aeropuertos, grafo)

def busqueda_profundidad(inicio: int, visitados: List[bool], grafo: Grafo) -> List[int]:
    """Realiza una búsqueda en profundidad (DFS) en el grafo."""
    pila = [inicio]
    componente = []

    while pila:
        vertice = pila.pop()
        if not visitados[vertice]:
            visitados[vertice] = True
            componente.append(vertice)
            pila.extend(vecino for vecino, _ in grafo[vertice] if not visitados[vecino])

    return componente

def analizar_conectividad(grafo: Grafo) -> List[List[int]]:
    """Analiza la conectividad del grafo y encuentra las componentes conexas."""
    n = len(grafo)
    visitados = [False] * n
    componentes = []

    for v in range(n):
        if not visitados[v]:
            componente = busqueda_profundidad(v, visitados, grafo)
            componentes.append(componente)

    return componentes

def arbol_expansion_minima_prim(grafo: Grafo, inicio: int) -> float:
    """Aplica el algoritmo de Prim para encontrar el árbol de expansión mínima (MST)."""
    n = len(grafo)
    visitados = [False] * n
    peso_total = 0
    visitados[inicio] = True
    candidatos = [(peso, inicio, vecino) for vecino, peso in grafo[inicio]]

    while candidatos and sum(visitados) < n:
        peso, _, v = min(candidatos)
        if not visitados[v]:
            peso_total += peso
            visitados[v] = True
            candidatos.extend((w, v, u) for u, w in grafo[v] if not visitados[u])
        candidatos = [c for c in candidatos if not visitados[c[2]]]

    return peso_total

def calcular_mst_para_componentes(grafo: Grafo, componentes: List[List[int]]) -> None:
    """Calcula y muestra el MST para cada componente conexa del grafo."""
    for i, componente in enumerate(componentes):
        if len(componente) > 1:
            inicio = componente[0]
            peso_total = arbol_expansion_minima_prim(grafo, inicio)
            print(f"Componente {i + 1}:")
            print(f" - Peso total del MST: {peso_total:.2f} km")
        else:
            print(f"Componente {i + 1} tiene solo un vértice, no se puede formar un MST.")
    print()

def generar_mapa_aeropuertos(codigos_aeropuertos: List[str], coordenadas_aeropuertos: List[Coordenada]) -> None:
    """Genera un mapa con los aeropuertos y lo guarda como un archivo PNG."""
    plt.figure(figsize=(20, 15))  # Aumentado el tamaño para mejor resolución

    for codigo, (lat, lon) in zip(codigos_aeropuertos, coordenadas_aeropuertos):
        plt.plot(lon, lat, 'bo', markersize=5)
        plt.text(lon + 0.5, lat + 0.5, codigo, fontsize=8, alpha=0.7)

    plt.title("Mapa de Aeropuertos", fontsize=20)
    plt.xlabel("Longitud", fontsize=14)
    plt.ylabel("Latitud", fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.7)

    # Ajustar los límites del mapa para incluir todos los aeropuertos
    plt.xlim(min(lon for _, lon in coordenadas_aeropuertos) - 5, max(lon for _, lon in coordenadas_aeropuertos) + 5)
    plt.ylim(min(lat for lat, _ in coordenadas_aeropuertos) - 5, max(lat for lat, _ in coordenadas_aeropuertos) + 5)

    # Guardar el mapa como archivo PNG
    plt.savefig(RUTA_SALIDA_MAPA, dpi=300, bbox_inches='tight')
    plt.close()  # Cerrar la figura para liberar memoria

    print(f"Mapa guardado como {RUTA_SALIDA_MAPA}")

def main():
    # Cargar datos
    codigos_aeropuertos, coordenadas_aeropuertos, grafo, info_aeropuertos = cargar_datos_aeropuertos(RUTA_ARCHIVO_CSV)

    while True:
        print("Opciones:")
        print("----------------------------------------")
        print("1. Mostrar información de un aeropuerto y los 10 más lejanos")
        print("2. Mostrar camino mínimo entre dos aeropuertos")
        print("3. Mostrar cantidad de componentes y cantidad de vertices")
        print("4. Determinar MST")
        print("5. Mostrar mapa")
        print("6. Salir")
        print("----------------------------------------")

        opcion = input("Seleccione una opción: ")
        print("----------------------------------------")

        if opcion == '1':
            codigo_inicio = input("Ingrese el código del aeropuerto de inicio: ")
            mostrar_aeropuertos_lejanos_misma_componente(codigo_inicio, codigos_aeropuertos, info_aeropuertos, grafo)
            print("----------------------------------------")
        elif opcion == '2':
            codigo_inicio = input("Ingrese el código del aeropuerto de inicio: ")
            codigo_fin = input("Ingrese el código del aeropuerto de destino: ")
            print("----------------------------------------")
            mostrar_camino_minimo(codigo_inicio, codigo_fin, codigos_aeropuertos, info_aeropuertos, grafo)
            print("----------------------------------------")
        elif opcion == '3':
            # Analizar conectividad
            componentes = analizar_conectividad(grafo)
            # Mostrar resultados de conectividad
            if len(componentes) == 1:
                print("El grafo es conexo.")
            else:
                print(f"El grafo no es conexo. Tiene {len(componentes)} componentes.")
                print("----------------------------------------")
                for i, comp in enumerate(componentes):
                    print(f"Componente {i + 1}: {len(comp)} vértices")
                    print("----------------------------------------")

        elif opcion == '4':
            # Analizar conectividad
            componentes = analizar_conectividad(grafo)
            # Calcular MST para cada componente
            calcular_mst_para_componentes(grafo, componentes)
            print("----------------------------------------")

        elif opcion == '5':
            # Generar mapa
            generar_mapa_aeropuertos(codigos_aeropuertos, coordenadas_aeropuertos)
            print("----------------------------------------")

        elif opcion == '6':
            print("Gracias por usar el programa.")
            break
        else:
            print("Opción no válida. Por favor, intente de nuevo.")

if __name__ == "__main__":
    main()


Opciones:
----------------------------------------
1. Mostrar información de un aeropuerto y los 10 más lejanos
2. Mostrar camino mínimo entre dos aeropuertos
3. Mostrar cantidad de componentes y cantidad de vertices
4. Determinar MST
5. Mostrar mapa
6. Salir
----------------------------------------
Seleccione una opción: 4
----------------------------------------
Componente 1:
 - Peso total del MST: 1197584.25 km
Componente 2:
 - Peso total del MST: 1598.90 km
Componente 3:
 - Peso total del MST: 787.25 km
Componente 4:
 - Peso total del MST: 1441.06 km
Componente 5:
 - Peso total del MST: 255.94 km
Componente 6:
 - Peso total del MST: 98.79 km
Componente 7:
 - Peso total del MST: 37.83 km

----------------------------------------
Opciones:
----------------------------------------
1. Mostrar información de un aeropuerto y los 10 más lejanos
2. Mostrar camino mínimo entre dos aeropuertos
3. Mostrar cantidad de componentes y cantidad de vertices
4. Determinar MST
5. Mostrar mapa
6. Sali